In [1]:
%cd ..

/home/feodor/Jupyter/depth-poset


/home/feodor/Jupyter/jupyter-env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os

import numpy as np
import pandas as pd
import pickle as pkl

from tqdm.notebook import tqdm

# Load and Format the Data

In [3]:
# define file paths
directory = "results/scores-on-random-alpha-complexes"
paths = np.sort([f'{directory}/{f}' for f in os.listdir(directory)])

# load them
ls = []
for path in tqdm(paths, total=len(paths)):
    with open(path, 'rb') as file:
        ls.append(pkl.load(file))

  0%|          | 0/59 [00:00<?, ?it/s]

In [4]:
pd.DataFrame(ls)

,n,dim,points,depth poset,number_of_nodes,number_of_minimal_nodes,number_of_maximal_nodes,height,width,minimum_maximal_chain,avarage_maximal_chain,incomparable_number,incestors_number,incestors_height,incestors_width,successors_number,successors_height,successors_width
0,4,1,"[[0.9854914071953744], [0.9340817602751083], [...",<src.depth.DepthPoset object at 0x7579075335c0>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1.5, 0: 1.5}","{ShallowPair([0.0000, 0.0493], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0493], dim=0): {0: 1},..."
1,4,1,"[[0.10486377460556096], [0.05147769242687872],...",<src.depth.DepthPoset object at 0x7578f8b1b8f0>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1.5, 0: 1.5}","{ShallowPair([0.0000, 0.0007], dim=0): {0: 2},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0007], dim=0): {0: 0},..."
2,4,1,"[[0.27603689782521335], [0.5513562992058008], ...",<src.depth.DepthPoset object at 0x7578f8b1b950>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1.5, 0: 1.5}","{ShallowPair([0.0000, 0.0012], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0012], dim=0): {0: 0},..."
3,4,1,"[[0.17000914651024235], [0.9752733863183506], ...",<src.depth.DepthPoset object at 0x7578f8b1ba40>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1.5, 0: 1.5}","{ShallowPair([0.0000, 0.1621], dim=0): {0: 2},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.1621], dim=0): {0: 0},..."
4,4,1,"[[0.01107290556871765], [0.9057618724888618], ...",<src.depth.DepthPoset object at 0x75790401c650>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 2.0, 0: 2.0}","{ShallowPair([0.0000, 0.0026], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0026], dim=0): {0: 0},..."
5,4,1,"[[0.35718511263219666], [0.3206138085574093], ...",<src.depth.DepthPoset object at 0x7578f8b44050>,"{'full': 3, 0: 3}","{'full': 2, 0: 2}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 2, 0: 2}","{'full': 1, 0: 1}","{'full': 1.5, 0: 1.5}","{ShallowPair([0.0000, 0.0003], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 1},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 0},...","{ShallowPair([0.0000, 0.0003], dim=0): {0: 1},..."
6,4,1,"[[0.5297888914346025], [0.8258927767565